# Homework 2 - Computational Thinking and Drawing with Turtle

## Before you begin

### Submission
Before you start working on this Homework:

1. Press **File->Save a copy in Drive** from the menu. This will save your own copy of this notebook

2. Follow the steps below and ensure you run the cells in order from the top. (You'll have to ensure you run any cells above this one too)

3. You should get into the habit of saving your file occassionally as you work

4. Certain cells will check your work against the expected answer, when you run these cells you'll know if your work is correct.

5. After all the changes are done and progress is saved **File->Download->Download.ipynb** and then upload the .ipynb file to moodle

In [ ]:
# The following code will download the required public tests for autograding
# Run this cell before the below set-up cell

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

tests_path = "https://github.com/andrewgodbout/CS1910-Test/raw/refs/heads/main/HW/HW2/tests.zip"


download(tests_path)
!unzip -o tests.zip

In [ ]:
#@title Run your setup...
#@markdown When you run this block you will either see ✅ SUCCESS or ❌ ERROR <br> letting you know if the library has run correctly. <br><br>
#@markdown If there is an error in this block please email your instructor for help.

try:
    # Standard library imports
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import ipywidgets as widgets
    import IPython.display as ipd

    #otter for autograding
    !pip install otter-grader==5.5.0
    import otter
    grader = otter.Notebook(colab=True)

    # Turtle library for graphics
    !pip install ColabTurtlePlus
    import ColabTurtlePlus.Turtle as turtle

    # Lab Form Library - Reusable Components (Widget-Only)
    class LabFormLibrary:
        """Reusable form components for interactive lab exercises"""

        @staticmethod
        def create_info_form(title, fields, default_values=None):
            """
            Create a form for collecting student information

            Args:
                title (str): Form title
                fields (list): List of dicts with 'name', 'label', 'placeholder' keys
                default_values (list, optional): List of default values for each field
            """
            field_widgets = {}
            field_containers = []

            for i, field in enumerate(fields):
                # Get placeholder value if provided
                placeholder_value = field.get('placeholder', 'Enter value')
                if default_values and i < len(default_values):
                    placeholder_value = str(default_values[i])

                widget = widgets.Text(
                    placeholder=placeholder_value,  # ← UPDATED LINE
                    description=field['label'],
                    style={'description_width': '150px'},
                    layout=widgets.Layout(width='400px', margin='0 0 5px 0')
                )
                field_widgets[field['name']] = widget
                field_containers.append(widget)

            submit_button = widgets.Button(
                description='Submit Information',
                button_style='success',
                layout=widgets.Layout(width='200px')
            )

            edit_button = widgets.Button(
                description='Edit Information',
                button_style='warning',
                layout=widgets.Layout(width='200px')
            )

            output_area = widgets.Output()

            def submit_info(button):
                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📝 Submitted {title}:")
                    print("=" * 40)
                    for field in fields:
                        value = field_widgets[field['name']].value
                        print(f"{field['label']}: {value}")
                    print("=" * 40)
                    print("✅ Information saved! You can edit it anytime using the 'Edit Information' button.")

                form_container.layout.display = 'none'
                submit_button.layout.display = 'none'
                edit_button.layout.display = 'block'

            def edit_info(button):
                form_container.layout.display = 'block'
                submit_button.layout.display = 'block'
                edit_button.layout.display = 'none'

                with output_area:
                    ipd.clear_output(wait=True)
                    print("📝 Edit mode activated - make your changes and click Submit again.")

            submit_button.on_click(submit_info)
            edit_button.on_click(edit_info)
            edit_button.layout.display = 'none'

            title_widget = widgets.Label(
                value=title,
                layout=widgets.Layout(margin='0 0 15px 0')
            )

            form_container = widgets.VBox([title_widget] + field_containers)
            button_container = widgets.HBox([submit_button, edit_button])
            all_content = widgets.VBox([form_container, button_container, output_area])

            def display_form():
                ipd.display(all_content)

            def get_data():
                return {field['name']: field_widgets[field['name']].value for field in fields}

            return display_form, get_data

        @staticmethod
        def create_question_form(title, questions, answer_key=None, default_values=None, data_lst=None):
            """
            Create a question and answer form with proper text wrapping

            Args:
                title (str): Form title
                questions (list): List of dicts with 'number', 'question', 'type' keys
                answer_key (list, optional): List of model answers for reveal
                default_values (list, optional): List of default values for each question
            """
            answer_widgets = []
            question_containers = []
            _answers = []
            _save_answers = data_lst is not None

            for i, q in enumerate(questions):
                # Use HTML widget for better text wrapping
                question_html = widgets.HTML(
                    value=f"<div style='margin: 15px 0 5px 0; word-wrap: break-word; line-height: 1.4;'><b>Question {q['number']}:</b> {q['question']}</div>",
                    layout=widgets.Layout(width='95%', max_width='800px')
                )

                # Get placeholder value if provided
                placeholder_value = 'Enter your answer here...'
                if default_values and i < len(default_values):
                    placeholder_value = str(default_values[i])

                if q.get('type', 'text') == 'textarea':
                    widget = widgets.Textarea(
                        placeholder=placeholder_value,  # ← UPDATED LINE
                        layout=widgets.Layout(width='95%', height='80px', max_width='800px', margin='0 0 10px 0'),
                        description='',
                        style={'description_width': '0px'}
                    )
                else:
                    widget = widgets.Text(
                        placeholder=placeholder_value,  # ← UPDATED LINE
                        layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 10px 0'),
                        description='',
                        style={'description_width': '0px'}
                    )

                answer_widgets.append(widget)
                _answers.append("")
                question_container = widgets.VBox([question_html, widget],
                    layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 10px 0'))
                question_containers.append(question_container)


            submit_button = widgets.Button(
                description='Submit All Answers',
                button_style='success',
                layout=widgets.Layout(width='200px', margin='10px 5px 0 0')
            )

            edit_button = widgets.Button(
                description='Edit Answers',
                button_style='warning',
                layout=widgets.Layout(width='150px', margin='10px 5px 0 0')
            )

            reveal_button = widgets.Button(
                description='Reveal Answers',
                button_style='info',
                layout=widgets.Layout(width='150px', margin='10px 0 0 0')
            )

            output_area = widgets.Output()

            def submit_answers(button):
                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📝 Submitted {title}:")
                    print("=" * 80)
                    _answers.clear()
                    if _save_answers:
                        data_lst.clear()
                        
                    for i, (q, widget) in enumerate(zip(questions, answer_widgets)):
                        answer = widget.value.strip() if widget.value.strip() else "(no answer provided)"
                        print(f"\nQuestion {q['number']}:")
                        print(f"Q: {q['question']}")
                        print(f"A: {answer}")
                        _answers.append(answer)
                        if _save_answers:
                            data_lst.append(answer)
                        print("-" * 60)

                    print("\n✅ All answers saved! You can edit them anytime using the 'Edit Answers' button.")

                form_container.layout.display = 'none'
                submit_button.layout.display = 'none'
                edit_button.layout.display = 'block'
                if answer_key:
                    reveal_button.layout.display = 'block'
                

            def edit_answers(button):
                form_container.layout.display = 'block'
                submit_button.layout.display = 'block'
                edit_button.layout.display = 'none'
                reveal_button.layout.display = 'none'

                with output_area:
                    ipd.clear_output(wait=True)
                    print("📝 Edit mode activated - make your changes and click 'Submit All Answers' again.")

            def reveal_answers(button):
                if not answer_key:
                    return

                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📚 ANSWER KEY - {title}:")
                    print("=" * 100)

                    for i, (q, widget) in enumerate(zip(questions, answer_widgets)):
                        student_answer = widget.value.strip() if widget.value.strip() else "(no answer provided)"
                        correct_answer = answer_key[i] if i < len(answer_key) else "No model answer provided"

                        print(f"\nQuestion {q['number']}:")
                        print(f"Q: {q['question']}")
                        print(f"Your Answer: {student_answer}")
                        print(f"Model Answer: {correct_answer}")
                        print("-" * 100)

                    print("\n📝 Note: These are model answers. Your responses may vary in wording")
                    print("while still demonstrating correct understanding of the concepts.")
                    print("For autograding you should try to match exactly the model answers")

            submit_button.on_click(submit_answers)
            edit_button.on_click(edit_answers)
            reveal_button.on_click(reveal_answers)

            edit_button.layout.display = 'none'
            reveal_button.layout.display = 'none'

            title_widget = widgets.Label(value=title, layout=widgets.Layout(margin='0 0 15px 0'))

            form_container = widgets.VBox([title_widget] + question_containers,
                layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 0 0'))

            button_list = [submit_button, edit_button]
            if answer_key:
                button_list.append(reveal_button)

            button_container = widgets.HBox(button_list,
                layout=widgets.Layout(width='95%', max_width='800px', margin='10px 0 0 0'))

            all_content = widgets.VBox([form_container, button_container, output_area],
                layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 0 0'))

            def get_data():
                return _answers
            
            def display_form():
                ipd.display(all_content)

            return display_form, get_data

        @staticmethod
        def create_prediction_table(title, code_snippets, expected_answers=None, default_values=None):
            """
            Create a prediction table for code output with fixed vertical output formatting

            Args:
                title (str): Table title
                code_snippets (list): List of code strings to predict
                expected_answers (list, optional): List of expected outputs for reveal
                default_values (list, optional): List of default predictions for each code snippet (used as placeholder text)
            """
            table_widgets = []
            table_rows = []

            title_widget = widgets.Label(value=title, layout=widgets.Layout(margin='0 0 10px 0'))
            instruction_widget = widgets.Label(
                value="Enter your prediction for what each line of Python code will output:",
                layout=widgets.Layout(margin='0 0 15px 0')
            )

            code_header = widgets.Label(
                value="Python Code",
                layout=widgets.Layout(width='50%', margin='0 0 5px 0')
            )
            prediction_header = widgets.Label(
                value="Your Prediction",
                layout=widgets.Layout(width='50%', margin='0 0 5px 0')
            )

            header_row = widgets.HBox([code_header, prediction_header],
                layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 10px 0'))

            for i, code in enumerate(code_snippets):
                code_display = widgets.Label(
                    value=code,
                    layout=widgets.Layout(width='50%', margin='0 5px 0 0')
                )

                # Get placeholder value if provided
                placeholder_value = 'Enter output or leave blank if none'
                if default_values and i < len(default_values):
                    placeholder_value = str(default_values[i])

                prediction_widget = widgets.Text(
                    placeholder=placeholder_value,
                    layout=widgets.Layout(width='45%', margin='0 0 0 0'),
                    description='',
                    style={'description_width': '0px'}
                )

                table_widgets.append(prediction_widget)

                row_container = widgets.HBox([code_display, prediction_widget],
                    layout=widgets.Layout(width='95%', max_width='800px', margin='2px 0 2px 0'))

                table_rows.append(row_container)

            submit_button = widgets.Button(
                description='Submit Answers',
                button_style='success',
                layout=widgets.Layout(width='150px', margin='10px 5px 0 0')
            )

            edit_button = widgets.Button(
                description='Edit Answers',
                button_style='warning',
                layout=widgets.Layout(width='150px', margin='10px 5px 0 0')
            )

            reveal_button = widgets.Button(
                description='Reveal Answers',
                button_style='info',
                layout=widgets.Layout(width='150px', margin='10px 0 0 0')
            )

            output_area = widgets.Output()

            def submit_answers(button):
                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📝 Submitted {title}:")
                    print("=" * 80)

                    # FIXED: Use vertical format instead of table format
                    for i, (code, widget) in enumerate(zip(code_snippets, table_widgets)):
                        answer = widget.value if widget.value else "(no output)"

                        print(f"Code: {code}")
                        print(f"Your Answer: {answer}")
                        print("-" * 40)

                    print("✅ Answers saved! You can edit them anytime using the 'Edit Answers' button.")

                table_container.layout.display = 'none'
                submit_button.layout.display = 'none'
                edit_button.layout.display = 'block'
                if expected_answers:
                    reveal_button.layout.display = 'block'

            def edit_answers(button):
                table_container.layout.display = 'block'
                submit_button.layout.display = 'block'
                edit_button.layout.display = 'none'
                reveal_button.layout.display = 'none'

                with output_area:
                    ipd.clear_output(wait=True)
                    print("📝 Edit mode activated - make your changes and click Submit again.")

            def reveal_answers(button):
                if not expected_answers:
                    return

                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📚 ANSWER KEY - {title} Comparison:")
                    print("=" * 80)

                    # FIXED: Use vertical format with validation
                    for i, (code, widget) in enumerate(zip(code_snippets, table_widgets)):
                        student_answer = widget.value if widget.value else "(no output)"
                        correct_answer = expected_answers[i] if i < len(expected_answers) and expected_answers[i] else "(no output)"

                        print(f"Code: {code}")
                        print(f"Your Answer: {student_answer}")
                        print(f"Correct Answer: {correct_answer}")

                        # Add validation
                        is_correct = student_answer.strip().lower() == correct_answer.strip().lower()
                        result = "✅ Correct" if is_correct else "❌ Incorrect"
                        print(f"Result: {result}")
                        print("-" * 50)

                    print("📝 Note: Remember that assignment statements and some expressions don't produce output.")
                    print("SyntaxError occurs when Python can't understand the code syntax.")

            submit_button.on_click(submit_answers)
            edit_button.on_click(edit_answers)
            reveal_button.on_click(reveal_answers)

            edit_button.layout.display = 'none'
            reveal_button.layout.display = 'none'

            table_container = widgets.VBox([title_widget, instruction_widget, header_row] + table_rows,
                layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 0 0'))

            button_list = [submit_button, edit_button]
            if expected_answers:
                button_list.append(reveal_button)

            button_container = widgets.HBox(button_list,
                layout=widgets.Layout(width='95%', max_width='800px', margin='10px 0 0 0'))

            all_content = widgets.VBox([table_container, button_container, output_area],
                layout=widgets.Layout(width='95%', max_width='800px', margin='0 0 0 0'))

            def display_form():
                ipd.display(all_content)

            return display_form

        @staticmethod
        def create_validation_form(title, data_rows, headers, correct_answers, instructions="", default_values=None):
            """
            Create a form with validation (correct/incorrect checking)

            Args:
                title (str): Form title
                data_rows (list): List of dicts with fixed data for each row
                headers (list): List of column headers
                correct_answers (list): List of lists with correct answers for validation
                instructions (str): Additional instructions
                default_values (list, optional): List of lists with default values for input columns
            """
            input_widgets = []
            table_rows = []

            title_widget = widgets.Label(value=title, layout=widgets.Layout(margin='0 0 10px 0'))

            instruction_widget = widgets.Label(
                value=instructions,
                layout=widgets.Layout(margin='0 0 15px 0')
            ) if instructions else None

            num_cols = len(headers)
            col_width = f"{90//num_cols}%"

            header_widgets = []
            for header in headers:
                header_label = widgets.Label(
                    value=header,
                    layout=widgets.Layout(width=col_width, margin='0 0 5px 0')
                )
                header_widgets.append(header_label)

            header_row = widgets.HBox(header_widgets,
                layout=widgets.Layout(width='95%', max_width='600px', margin='5px 0 10px 0'))

            for i, row_data in enumerate(data_rows):
                row_widgets = []
                row_inputs = []
                input_idx = 0

                for j, header in enumerate(headers):
                    if header in row_data:
                        widget = widgets.Label(
                            value=str(row_data[header]),
                            layout=widgets.Layout(width=col_width, margin='0 0 2px 0')
                        )
                        row_widgets.append(widget)
                    else:
                        # Get placeholder value if provided
                        placeholder_value = 'True/False'
                        if default_values and i < len(default_values) and input_idx < len(default_values[i]):
                            placeholder_value = str(default_values[i][input_idx])

                        input_widget = widgets.Text(
                            placeholder=placeholder_value,  # ← UPDATED LINE
                            layout=widgets.Layout(width=col_width, margin='0 0 2px 0'),
                            description='',
                            style={'description_width': '0px'}
                        )
                        row_widgets.append(input_widget)
                        row_inputs.append(input_widget)
                        input_idx += 1

                input_widgets.append(row_inputs)

                row = widgets.HBox(row_widgets,
                    layout=widgets.Layout(width='95%', max_width='600px', margin='2px 0 2px 0'))

                table_rows.append(row)

            submit_button = widgets.Button(
                description='Submit Table',
                button_style='success',
                layout=widgets.Layout(width='150px', margin='10px 5px 0 0')
            )

            edit_button = widgets.Button(
                description='Edit Table',
                button_style='warning',
                layout=widgets.Layout(width='120px', margin='10px 5px 0 0')
            )

            reveal_button = widgets.Button(
                description='Reveal Answers',
                button_style='info',
                layout=widgets.Layout(width='150px', margin='10px 0 0 0')
            )

            output_area = widgets.Output()

            def submit_table(button):
                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📝 Submitted {title}:")
                    print("=" * 50)

                    header_str = " | ".join([f"{h:<8}" for h in headers])
                    print(header_str)
                    print("-" * 50)

                    for i, (row_data, row_inputs) in enumerate(zip(data_rows, input_widgets)):
                        row_values = []
                        input_idx = 0
                        for header in headers:
                            if header in row_data:
                                row_values.append(f"{row_data[header]:<8}")
                            else:
                                answer = row_inputs[input_idx].value.strip() if row_inputs[input_idx].value.strip() else "(blank)"
                                row_values.append(f"{answer:<8}")
                                input_idx += 1

                        print(" | ".join(row_values))

                    print("=" * 50)
                    print("✅ Table submitted!")

                table_container.layout.display = 'none'
                submit_button.layout.display = 'none'
                edit_button.layout.display = 'block'
                reveal_button.layout.display = 'block'

            def edit_table(button):
                table_container.layout.display = 'block'
                submit_button.layout.display = 'block'
                edit_button.layout.display = 'none'
                reveal_button.layout.display = 'none'

                with output_area:
                    ipd.clear_output(wait=True)
                    print("📝 Edit mode activated - make your changes and click 'Submit Table' again.")

            def reveal_answers(button):
                with output_area:
                    ipd.clear_output(wait=True)
                    print(f"📚 ANSWER KEY - {title} Comparison:")
                    print("=" * 120)

                    comparison_headers = []
                    for header in headers:
                        if header not in data_rows[0]:
                            comparison_headers.extend([f"Your {header}", f"Correct {header}"])
                        else:
                            comparison_headers.append(header)
                    comparison_headers.append("Result")

                    header_str = " | ".join([f"{h:<12}" for h in comparison_headers])
                    print(header_str)
                    print("-" * 120)

                    all_correct = True

                    for i, (row_data, row_inputs) in enumerate(zip(data_rows, input_widgets)):
                        row_values = []
                        input_idx = 0
                        row_correct = True

                        for j, header in enumerate(headers):
                            if header in row_data:
                                row_values.append(f"{row_data[header]:<12}")
                            else:
                                student_answer = row_inputs[input_idx].value.strip() if row_inputs[input_idx].value.strip() else "(blank)"
                                correct_answer = correct_answers[i][input_idx] if i < len(correct_answers) and input_idx < len(correct_answers[i]) else "Unknown"

                                is_correct = student_answer.lower() == correct_answer.lower()
                                if not is_correct:
                                    row_correct = False
                                    all_correct = False

                                row_values.extend([f"{student_answer:<12}", f"{correct_answer:<12}"])
                                input_idx += 1

                        result = "✅ Correct" if row_correct else "❌ Incorrect"
                        row_values.append(f"{result:<12}")

                        print(" | ".join(row_values))

                    print("=" * 120)

                    if all_correct:
                        print("🎉 Perfect! You got all answers correct!")
                    else:
                        print("📝 Review the incorrect answers above and try to understand the logic.")

            submit_button.on_click(submit_table)
            edit_button.on_click(edit_table)
            reveal_button.on_click(reveal_answers)

            edit_button.layout.display = 'none'
            reveal_button.layout.display = 'none'

            container_widgets = [title_widget]
            if instruction_widget:
                container_widgets.append(instruction_widget)
            container_widgets.append(header_row)
            container_widgets.extend(table_rows)

            table_container = widgets.VBox(container_widgets,
                layout=widgets.Layout(width='95%', max_width='700px', margin='0 0 0 0'))

            button_container = widgets.HBox([submit_button, edit_button, reveal_button],
                layout=widgets.Layout(width='95%', max_width='700px', margin='10px 0 0 0'))

            all_content = widgets.VBox([table_container, button_container, output_area],
                layout=widgets.Layout(width='95%', max_width='700px', margin='0 0 0 0'))

            def display_form():
                ipd.display(all_content)

            return display_form

        @staticmethod
        def add_educational_context(title, sections):
            """
            Add educational context section with formatted content

            Args:
                title (str): Main title for the educational section
                sections (list): List of dicts with 'heading' and 'content' keys
            """
            from IPython.display import Markdown, display

            # Build the markdown content
            markdown_content = f"### {title}\n\n"

            for section in sections:
                # Add section heading with emoji if provided
                heading = section['heading']
                if 'emoji' in section:
                    heading = f"{section['emoji']} **{heading}**"
                else:
                    heading = f"#### **{heading}:**"

                markdown_content += f"{heading}\n"
                markdown_content += f"{section['content']}\n\n"

            # Display the formatted content
            display(Markdown(markdown_content))

        @staticmethod
        def add_quick_context(content):
            """
            Quick way to add educational context with just content

            Args:
                content (str): Markdown-formatted content to display
            """
            from IPython.display import Markdown, display
            display(Markdown(content))

    # Initialize the library for use in HW exercises
    forms = LabFormLibrary()

    print("✅ SUCCESS: All libraries imported successfully!")
    print("✅ HW Form Library loaded and ready!")
    print("📚 Available form types:")
    print("   • forms.create_info_form() - Information collection")
    print("   • forms.create_question_form() - Q&A with model answers")
    print("   • forms.create_prediction_table() - Code output prediction")
    print("   • forms.create_validation_form() - Truth tables with validation")
    print("   • forms.add_educational_context() - Structured learning context")
    print("   • forms.add_quick_context() - Simple markdown context")
    print("🆕 NEW: All forms now support optional default values!")
    print("You're ready to start the HW!")

except ImportError as e:
    print("❌ ERROR: There was a problem importing the required libraries.")
    print("\n🆘 PLEASE EMAIL YOUR INSTRUCTOR FOR HELP")
    print("   They will help you resolve this import issue.")
    print("\n" + "="*50)
    print("🔍 Technical Details (click triangle below to expand):")

    from IPython.display import HTML, display
    display(HTML(f"""
    <details>
    <summary><strong>Click here if you're curious about the error details</strong></summary>
    <pre style="background-color: #f5f5f5; padding: 10px; border-radius: 5px;">
    ImportError: {e}
    </pre>
    <p><em>Your instructor can use these details to help diagnose the problem.</em></p>
    </details>
    """))

except Exception as e:
    print("❌ ERROR: Something unexpected happened during setup.")
    print("\n🆘 PLEASE EMAIL YOUR INSTRUCTOR FOR HELP")
    print("   Show them this error message.")
    print("\n" + "="*50)
    print("🔍 Technical Details (click triangle below to expand):")

    from IPython.display import HTML, display
    display(HTML(f"""
    <details>
    <summary><strong>Click here if you're curious about the error details</strong></summary>
    <pre style="background-color: #f5f5f5; padding: 10px; border-radius: 5px;">
    Error: {e}
    Error Type: {type(e).__name__}
    </pre>
    <p><em>Your instructor can use these details to help diagnose the problem.</em></p>
    </details>
    """))

In [ ]:
#@title Student Information Form
#@markdown Enter your student ID in the form

# STEP 1: Define your data
title = "Homework 2 Thinking and Drawing with Turtle"
fields = [
    {
        'name': 'field_variable_name',
        'label': 'Enter your student id:',
        'placeholder': 'Student ID'
    },
    # Add more fields as needed
]

# STEP 2: Optional default values (will appear as placeholder text)
default_values_form1 = [
    "000000",
    # One value per field, or use None to skip
]

# STEP 3: Create and display the form
display_form, get_data = forms.create_info_form(title, fields, default_values_form1)
display_form()

# STEP 4: Optional - get the data later in your code
# student_data = get_data()
# print(f"Student name: {student_data['field_variable_name']}")


## Consider we want to use turtle draw to create a visual of an ice cream scoop

<img src="ice_cream.png" alt="ice cream drawing" width=100/>

A couple of functions that we haven't mentioned previously from ColabTurtlePlus:

1. You can make circles using the **circle(radius, extent, steps)** method

  - radius gives the radius of the circle (if positive the circle is drawn counterclock-wise and if negative then the circle is drawn clock-wise.

  - extent if extent is left off then the circle is 360 degrees (full circle), otherwise the circle is drawn up to extent degrees

  - steps if steps is provided the circle could be reduced to a regular polygon (for steps < 20).

2. the **getx()** and **gety()** and **position()** methods return the x or y or (x,y) coordinate giving the current position of the turtle on the screen

3. **towards(x,y)** returns the angle between the line from the turtle start position to the position specified by (x,y)

4. **distance(x,y)** returns the distance from the turtle's current position to specified position

5. **heading()** returns the direction (in degrees) the turtle is looking at

You might take a quick look at the Full ColabTurtlePlus documentation:
https://larryriddle.agnesscott.org/ColabTurtlePlus/documentation2.html

or feel free to revisit this at any time if you wanted more details about what turtles can do.


## Getting familiar Question Set 1

Before we dive into creating the ice cream drawing let's get more familiar with the ColabTurtlePlus environment and these new methods.

Run the below code and view the output. The code calls clearscreen and then asks for the turtle position by calling **position()** and which direction the turtle is facing by calling **heading()** the results of those function calls are stored into variables.

Feel free to modify the below code however you see fit.

*Remember you can always view the original version of this file from the original homework link. If you've saved a copy of this homework in your drive, you will be editing your own version (without affecting the original).*

*Reminder to save your work as you go*

In [ ]:
#ensure you ran the Run your setup cell to ensure all libraries are installed
#run this code to see some basic turtle data

#initialize the turtle and print out some information about it
from ColabTurtlePlus.Turtle import *

#clear the screen and start a new drawing
clearscreen()

#what is the current position and direction of the turtle?
initial_position = position()
initial_heading = heading()

#print the variables
print("initial position:", initial_position)
print("initial heading:", initial_heading)


In [ ]:
#@title Turtle Draw Basic Question Set 1
#@markdown Consult the above code and output, or write your own to help answer some basic questions about turtle draw. Run this cell to see the Q1 Form.
# STEP 1: Define your questions
title = "Turtle Draw Basics"
questions_basic1 = [
    {
        "number": 1,
        "question": "After clearscreen() what is the (x,y) position of the turtle?",
        "type": "text"  # or "textarea" for longer answers
    },
    {
        "number": 2,
        "question": "After clearscreen() what direction (N,S,E,W) is the turtle facing?",
        "type": "text"
    },
    {
        "number": 3,
        "question":  "After clearscreen() what direction (in degrees) is the turtle facing?",
        "type": "text"
    },
    {
        "number": 4,
        "question":  "What is the variable name of the variable that stores the heading information?",
        "type": "text"
    }
]

# STEP 2: Define model answers (optional)
q1_answer_key = [
    "(0,0)",
    "E",
    "0",
    "initial_heading"
]

# STEP 3: Optional default values (appear as placeholder text)
default_values_basic1 = [
    "(x, y)",
    "look at the arrow direction to know which way the turtle is facing",
    "the coordinates match the unit circle",
    "match the name exactly"
    # Leave empty string "" for no hint
]

# STEP 3a: where to populate answers
q1_data = [x for x in default_values_basic1]

# STEP 4: Create and display the form
display_form, get_data_q1 = forms.create_question_form(title, questions_basic1, q1_answer_key, default_values_basic1, data_lst=q1_data)
display_form()

### AUTO GRADING Q1 - Basics

Run the below cell to ensure your above question about Turtle set-up basics will be graded correct with the autograding system. You can run any of the grader.check cells to see if you've got the questions correct.


_Points:_ 0

In [ ]:
grader.check("q1")

## Turtle Draw Basics Question Set 2

Continue below to ensure we all have knowledge of the basics of how turtle draw works.

Run the code block (edit however you like) and then answer the questions that follow. You can type the answers into the form and check your solutions, just make sure to run the code block with `grader.check ...` to ensure the autograding will also grade your answers correctly.

Reminder to save your work as you go


_Points:_ 0

In [ ]:
#ensure you ran the Run your setup cell to ensure all libraries are installed
#run this code to see more basics turtle movements

from ColabTurtlePlus.Turtle import *

#clear the screen and start a new drawing
clearscreen()

#move forward
forward(10)
print("move forward 10") 

first_pos = position()

#print out the (x,y) position
print("new position:", first_pos)

#face down the screen
print("rotate right by 90 degrees")
right(90)

#move forward by 15 more
print("move forward by 15")
forward(15)

second_pos = position()

#print the (x,y) location
print("new position:", second_pos)

#which direction am I heading
facing = heading()

#print new heading
print("new heading:", facing)

In [ ]:
#@title Turtle Draw Basics Q2
#@markdown Consult the above code and output, or write your own to help answer some basic questions about turtle draw. Run this cell to see the Q2 form.
# STEP 1: Define your questions
title = "Turtle Draw Basics"
questions = [
    {
        "number": 1,
        "question": "If the turtle is moving to the right, which of X or Y is changing",
        "type": "text"  # or "textarea" for longer answers
    },
    {
        "number": 2,
        "question":  "Following from question 1: is the value increasing or decreasing?",
        "type": "text"
    },
    {
        "number": 3,
        "question":  "If the turtle is traveling down, which of X or Y is changing?",
        "type": "text"
    },
    {
        "number": 4,
        "question":  "Following from question 3: is the value increasing or decreasing?",
        "type": "text"
    },
    {
        "number": 5,
        "question":  "Which direction (N,S,E,W) cooresponds with 270 degrees?",
        "type": "text"
    },
    {
        "number": 6,
        "question":  "Which direction (N,S,E,W) cooresponds with 90 degrees?",
        "type": "text"
    },
    {
        "number": 7,
        "question":  "Rotating the turtle by 90 degrees to the right is the equivalent of rotating the turtle by how many degrees to the left?",
        "type": "text"
    }
]

# STEP 2: Define model answers (optional)
q2_answer_key = [
    "X",
    "increasing",
    "Y",
    "decreasing",
    "S",
    "N",
    "270"
]

q2_data = ["" for _ in q2_answer_key]

# STEP 4: Create and display the form
display_form, get_data_q2 = forms.create_question_form(title, questions, q2_answer_key, data_lst=q2_data)
display_form()

In [ ]:
grader.check("q2")

# Ice Cream Development

## Question Set 3 

Consider that we have the following code to draw part of the ice cream:

<img src="partial_ice_cream.png" alt="ice cream drawing" width=100/>

Run the code, you should see a drawing like what we have above.

## Close the Scoop

The next step is to draw a line "closing" the bottom part of the second scoop (or drawing the top part of the cone). 

We want a "straight-line" across the bottom. In the ColabTurtlePlus documentation there is a **setheading(to_angle)** which will set the heading for the turtle to a specific angle. In this case we want to travel exactly east so this is 0 degrees. 

1. **In the below code near the bottom, add a line to set the heading for the turtle to 0 degrees.**

2. Draw the line forming the top of the cone (thus closing the second scoop). But how long should the line be? The **getx()** and **gety()** methods will return the current x and y location (respectively) of the turtle. **Modify the lines containing`right_x = 0 and right_y = 0 by replacing the zeros with getx() or gety()**

3. Now the x value of the position the turtle is occupying after drawing the right half of the bottom scoop is saved into `right_x` and similarily the y position in `right_y`. **Save the x and y positions of the Turtle after the left side of the bottom scoop is drawn into variables `left_x` and `left_y` just like in the prior step.**
   
4. **Compute and Draw the bottom line** Compute and save the length of the bottom line into a variable named `bottom_length` the `forward` function will draw the line. Hint: if `right_x` contains the position of the right side x coordinate and `left_x` contains the left side coordinate what formula gives the distance is between them?


Ensure your variables are named exactly as shown in the above as they will be evaluated as part of the grading.

Run your code once your are complete and ensure the line for the top of the cone is drawn correctly.

In [ ]:
from ColabTurtlePlus.Turtle import *

clearscreen()

#set a few variables
scoop_radius = 50
cone_height = 200
second_scoop_arc = 130

#draw the top scoop of the ice cream
circle(scoop_radius)

#draw the bottom partially visible scoop

#right half
circle(-scoop_radius, second_scoop_arc)

#STEP 2 replace the 0's with function calls
right_x = 0
right_y = 0

#face the opposite direction
left(180)

#left half
circle(scoop_radius, 2*second_scoop_arc)

#STEP 3 replace the 0's with function calls
left_x = 0
left_y = 0 

#STEP 1 set the heading to 0

#STEP 4 Draw the bottom line, replace the 0 with the appropriate value
#hint use the above variables to calculate the value.
bottom_length = 0
forward(bottom_length)




## Cone

## Question Set 4

Now we want to complete the cone. The bottom of the cone should be at the mid-point between `right_x` and `left_x`. This can be found by adding `right_x` and `left_x` and dividing by two. We want to point the turtle towards this bottom point and then send it there. Lastly we need to bring the turtle back up to the top left corner of the cone to complete the drawing.

1. In the below code `cone_bottom = right_x + left_x / 2` there is an error. Fix the error by adding parenthesis `()` to the statement.

2. Supposing we want the cone to be 200 units tall then the y value at the bottom of the cone is going to be 200 less than `right_y` (or similarily `left_y`). The `towards(x,y)` returns the angle pointing to a certain point (x,y). Under Step 2 in the code add the correct x and y values into the `towards` function call. You already calculated the x value in the above step (cone_bottom).

3. Because we have an angled line for the cone, it will not be 200 units long. Instead we can use the **distance** method to determine the appropriate distance (to save us some trigonometry calculations). **distance(x,y)** will return the distance from the turtle to a given (x,y) point. Use the same parameters you supplied in step 2, as parameters to distance.

4. In step 2 we calculated the angle to our bottom cone point, saved that angle in a variable and then called setheading to point the turtle towards that point. Use the same approach to point the turtle to the top left corner of the cone (hint: `left_x` and `left_y` from above could be useful).

5. The length of the line between the turtle and the top left corner is the same as the length between the top right and the turtle (step 3). Reuse the `cone_side_length` variable and more the turtle forward using the **forward** method.

**NOTE:**

*Normally we don't break up the code for a turtle draw across multiple code cells, but we wanted to improve focus and readability. So just remember to run the above code to draw the scoops followed by the below code. You should not run the below code 2 times in a row (it won't recreate the scoops before trying to draw the cone) Always run the above code first then the below code.*

In [ ]:
# code to create the cone 
# this code will continue following your above code and
# the drawing will continue in the above turtle drawing
# just note if you make a mistake you'll need to run the above code 
# again to reset the picture


# STEP 1: Add parenthesis to fix the below expression
cone_bottom_x = right_x + left_x / 2

# STEP 2: point the turtle towards the bottom of the cone
# add the correct x and y values (not 0,0) to the towards method for the bottom of the cone
# we want the bottom of the cone to be 200 below the top of the cone (hint: think about right_y)
angle_to_bottom = towards( 0, 0 )
setheading(angle_to_bottom)

# STEP 3: Add the correct x,y parameters to distance so we know 
#         how long to draw the cone sides, these are the same values you used in step 2
cone_side_length = distance(0, 0)
forward(cone_side_length)

#STEP 4: Similar to step 2 except we want to target the top left corner of the cone
#        Hint: we already have variables that we can use (no formulas required).
angle_to_top_left = towards(0, 0)
setheading(angle_to_top_left)

forward(cone_side_length)

## Checking your solutions - Q3 Scoop Coding

The below q3 check will examine your variables and solution for the Scoop task (question set 3) 


_Points:_ 2

In [ ]:
grader.check("q3")

### Cone Coding Q4

q4 check will examine some variables and solution for the cone aspect of the drawing.

_Points:_ 2

In [ ]:
grader.check("q4")

# Submission

- Ensure you run all the cells in order from the top to the bottom
- Check to ensure all the tests are passing
      - run each of the cells containing `grader.check` ...
- After running all the cells (top to bottom) and ensuring the tests are as you want **Save your file** to your local drive
- File->Download->Download.ipynb and then upload the .ipynb file to Moodle under Homework 2 submission